<a href="https://colab.research.google.com/github/PHANI-27/ML-PROJECTS/blob/main/OBJECT_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio

In [ ]:
#object detection
import gradio as gr
from transformers import AutoImageProcessor, AutoModelForObjectDetection
import torch
from PIL import Image
import torchvision.transforms as T

# Load model and processor
processor = AutoImageProcessor.from_pretrained("microsoft/table-transformer-detection")
model = AutoModelForObjectDetection.from_pretrained("microsoft/table-transformer-detection")

# Helper function to draw bounding boxes
def draw_boxes(image, boxes, labels, scores, threshold=0.8):
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    from io import BytesIO

    fig, ax = plt.subplots(1)
    ax.imshow(image)

    for box, label, score in zip(boxes, labels, scores):
        if score > threshold:
            x, y, w, h = box
            rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='red', facecolor='none')
            ax.add_patch(rect)
            ax.text(x, y, f"{label}: {score:.2f}", color="white", fontsize=8, bbox=dict(facecolor='red', alpha=0.5))

    buf = BytesIO()
    plt.axis('off')
    plt.savefig(buf, format='png')
    buf.seek(0)
    return Image.open(buf)

# Prediction function
def detect_tables(image):
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)

    # Process results
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.8)[0]

    boxes = results["boxes"].tolist()
    labels = [model.config.id2label[label_id] for label_id in results["labels"]]
    scores = results["scores"].tolist()

    result_image = draw_boxes(image, boxes, labels, scores)
    return result_image

# Gradio interface
gr.Interface(
    fn=detect_tables,
    inputs=gr.Image(type="pil"),
    outputs="image",
    title="Table Detection with Microsoft Table Transformer",
    description="Upload an image to detect tables using the Microsoft Table Transformer model."
).launch()
